<a href="https://colab.research.google.com/github/prithvivinodnair/long-read-aligner-benchmark/blob/main/notebooks/week1_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 1 Setup – Long-Read Aligner Benchmark

This notebook sets up the environment for the project:
- Clones the `long-read-aligner-benchmark` GitHub repo
- Installs Python dependencies
- Prepares folders for reference genome, annotations, and simulated reads
- Installs NanoSim-H for Nanopore-like read simulation
- Installs Minimap2 and runs a first test alignment

This work corresponds to Week 1 tasks:
- Set up environment
- Download/add genome + GFF
- Generate simulated reads
- Run first aligner


In [3]:
!git clone https://github.com/prithvivinodnair/long-read-aligner-benchmark.git
%cd long-read-aligner-benchmark

!ls -R


Cloning into 'long-read-aligner-benchmark'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 75 (delta 12), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 19.55 KiB | 9.77 MiB/s, done.
Resolving deltas: 100% (12/12), done.
/content/long-read-aligner-benchmark
.:
data  environment  notebooks  README.md  results  scripts

./data:
annotations  reference	simulated_reads

./data/annotations:

./data/reference:

./data/simulated_reads:

./environment:
requirements.txt  setup_instructions.md

./notebooks:
README.md

./results:
figures  logs

./results/figures:

./results/logs:

./scripts:
evaluate_accuracy.py  run_minimap2.sh  run_ngmlr.sh  run_winnowmap.sh


In [4]:
# Install Python dependencies listed in environment/requirements.txt
!pip install -r environment/requirements.txt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print("Python environment is ready.")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 63.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 64.5 MB/s eta 0:00:00
  Created wheel for pybedtools: filename=pybedtools-0.12.0-cp312-cp312-linux_x86_64.whl size=14340823 sha256=b077d31642dda668b3b4326121125dff1d38cadb75a927d5748d8e18deb698b9
  Stored in directory: /root/.cache/pip/wheels/ac/38/f2/960d79e44a92afc0d34a4727c856ce0149ac23c3dcda174356
Successfully built pybedtools
Python environment is ready.


In [5]:
import os

folders = [
    "data",
    "data/reference",
    "data/annotations",
    "data/simulated_reads",
    "results",
    "results/logs",
    "results/figures"
]

for f in folders:
    os.makedirs(f, exist_ok=True)

print("Data and results folders are ready.")
!ls -R


Data and results folders are ready.
.:
data  environment  notebooks  README.md  results  scripts

./data:
annotations  reference	simulated_reads

./data/annotations:

./data/reference:

./data/simulated_reads:

./environment:
requirements.txt  setup_instructions.md

./notebooks:
README.md

./results:
figures  logs

./results/figures:

./results/logs:

./scripts:
evaluate_accuracy.py  run_minimap2.sh  run_ngmlr.sh  run_winnowmap.sh


In [6]:
from google.colab import files
import shutil
import os

print("Please upload your reference FASTA file (e.g., chr22.fa.gz).")
uploaded = files.upload()  # choose chr22.fa.gz from your computer

for filename in uploaded.keys():
    dest = f"data/reference/{filename}"
    shutil.move(filename, dest)
    print(f"Moved {filename} -> {dest}")

print("\nReference folder contents:")
!ls -lh data/reference


Please upload your reference FASTA file (e.g., chr22.fa.gz).


Saving chr22.fa.gz to chr22.fa.gz
Moved chr22.fa.gz -> data/reference/chr22.fa.gz

Reference folder contents:
total 12M
-rw-r--r-- 1 root root 12M Nov 20 01:57 chr22.fa.gz


In [7]:
!ls -lh data/reference


total 12M
-rw-r--r-- 1 root root 12M Nov 20 01:57 chr22.fa.gz


In [8]:
!gunzip data/reference/chr22.fa.gz


In [9]:
!ls -lh data/reference


total 50M
-rw-r--r-- 1 root root 50M Nov 20 01:57 chr22.fa


In [13]:
# Install Biopython for FASTA handling
!pip install biopython

from Bio import SeqIO
import os
import random

# Locate the reference FASTA
ref_dir = "data/reference"
ref_files = [f for f in os.listdir(ref_dir) if f.endswith((".fa", ".fasta", ".fna"))]

if not ref_files:
    raise FileNotFoundError("No FASTA found in data/reference. Make sure chr22.fa is there.")

reference_fasta = os.path.join(ref_dir, ref_files[0])
print("Using reference FASTA:", reference_fasta)

# Read the first (and likely only) sequence
record = next(SeqIO.parse(reference_fasta, "fasta"))
genome_seq = str(record.seq)
genome_len = len(genome_seq)
print("Reference length:", genome_len)

# Simulation parameters
n_reads = 2000       # number of reads
read_len = 10000     # length of each read
error_rate = 0.05    # 5% bases mutated to introduce noise

out_path = "data/simulated_reads/chr22_simulated.fa"

def mutate_base(b):
    bases = ["A", "C", "G", "T"]
    bases = [x for x in bases if x != b.upper()]
    return random.choice(bases)

with open(out_path, "w") as out_f:
    for i in range(n_reads):
        # pick random start so the read fits within the chromosome
        start = random.randint(0, genome_len - read_len - 1)
        seq = list(genome_seq[start:start+read_len])

        # introduce random substitutions
        for j in range(len(seq)):
            if random.random() < error_rate and seq[j] in "ACGTacgt":
                seq[j] = mutate_base(seq[j])

        read_seq = "".join(seq)
        out_f.write(f">read_{i}_start{start}_len{read_len}\n")
        # wrap sequence to 80 chars per line
        for k in range(0, len(read_seq), 80):
            out_f.write(read_seq[k:k+80] + "\n")

print(f"Simulated {n_reads} long reads written to {out_path}")

!ls -lh data/simulated_reads


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 35.5 MB/s eta 0:00:00
Using reference FASTA: data/reference/chr22.fa
Reference length: 50818468
Simulated 2000 long reads written to data/simulated_reads/chr22_simulated.fa
total 20M
-rw-r--r-- 1 root root 20M Nov 20 05:59 chr22_simulated.fa
-rw-r--r-- 1 root root 315 Nov 20 05:54 nanosim_chr22.log


In [14]:
# Install Minimap2
!apt-get update -y
!apt-get install -y minimap2

# Check version
!minimap2 --version


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://cli.github.com/packages stable InRelease
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,149 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,830 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,470 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,532 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/universe am

In [15]:
import os

# Find simulated reads FASTA
sim_dir = "data/simulated_reads"
sim_files = [f for f in os.listdir(sim_dir) if f.endswith((".fa", ".fasta"))]

if not sim_files:
    raise FileNotFoundError("No simulated FASTA reads found in data/simulated_reads.")

reads_fasta = os.path.join(sim_dir, sim_files[0])
print("Using simulated reads:", reads_fasta)

# Find reference FASTA
ref_dir = "data/reference"
ref_files = [f for f in os.listdir(ref_dir) if f.endswith((".fa", ".fasta", ".fna"))]

if not ref_files:
    raise FileNotFoundError("No FASTA reference found in data/reference.")

reference_fasta = os.path.join(ref_dir, ref_files[0])
print("Using reference:", reference_fasta)

# Ensure results/logs exists
os.makedirs("results/logs", exist_ok=True)
sam_out = "results/logs/minimap2_week1_chr22_simulated.sam"

# Run Minimap2 in ONT mode (-x map-ont) and output SAM
!minimap2 -x map-ont -a {reference_fasta} {reads_fasta} > {sam_out}

print("\nMinimap2 alignment complete. Output SAM:", sam_out)
!head -n 10 {sam_out}


Using simulated reads: data/simulated_reads/chr22_simulated.fa
Using reference: data/reference/chr22.fa
[M::mm_idx_gen::1.749*1.00] collected minimizers
[M::mm_idx_gen::2.293*1.23] sorted minimizers
[M::main::2.293*1.23] loaded/built the index for 1 target sequence(s)
[M::mm_mapopt_update::2.390*1.22] mid_occ = 238
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1
[M::mm_idx_stat::2.459*1.21] distinct minimizers: 5012420 (87.89% are singletons); average occurrences: 1.472; average spacing: 6.888; total length: 50818468
[M::worker_pipeline::14.114*1.63] mapped 2000 sequences
[M::main] Version: 2.24-r1122
[M::main] CMD: minimap2 -x map-ont -a data/reference/chr22.fa data/simulated_reads/chr22_simulated.fa
[M::main] Real time: 14.132 sec; CPU: 23.055 sec; Peak RSS: 0.397 GB

Minimap2 alignment complete. Output SAM: results/logs/minimap2_week1_chr22_simulated.sam
@SQ	SN:chr22	LN:50818468
@PG	ID:minimap2	PN:minimap2	VN:2.24-r1122	CL:minimap2 -x map-ont -a data/reference/chr22.fa 

In [16]:
from google.colab import files

# Download reference FASTA
files.download("data/reference/chr22.fa")

# Download simulated reads
files.download("data/simulated_reads/chr22_simulated.fa")

# Download SAM alignment
files.download("results/logs/minimap2_week1_chr22_simulated.sam")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>